In [66]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from sklearn.gaussian_process import GaussianProcessRegressor
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BicScore

In [3]:
df =  pd.read_csv('gas_grid.csv')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,23,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,Unnamed: 0,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,...,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,Location,RowID,lat,long,lonlat,geometry,FID
0,0,1050242,E12,31236,Gas Leak (Natural and LP Gases),4/14/00,4/14/00,4/14/00 16:58,4/14/00 16:59,4/14/00 17:01,...,5,7,Inner Sunset,"(37.7633165969878, -122.458205988587)",001050242-E12,37.763317,-122.458206,"(-122.458205988587, 37.7633165969878)",POINT (-122.458205988587 37.7633165969878),286.0
1,1,1050319,E44,31311,Gas Leak (Natural and LP Gases),4/14/00,4/14/00,4/14/00 20:34,4/14/00 20:34,4/14/00 20:35,...,9,10,Visitacion Valley,"(37.7096032041108, -122.409888540797)",001050319-E44,37.709603,-122.409889,"(-122.409888540797, 37.7096032041108)",POINT (-122.409888540797 37.7096032041108),551.0
2,2,1070275,E17,31946,Gas Leak (Natural and LP Gases),4/16/00,4/16/00,4/16/00 20:38,4/16/00 20:39,4/16/00 20:39,...,10,10,Bayview Hunters Point,"(37.7291935618432, -122.374002352858)",001070275-E17,37.729194,-122.374002,"(-122.374002352858, 37.7291935618432)",POINT (-122.374002352858 37.7291935618432),731.0
3,3,1080336,E17,32305,Gas Leak (Natural and LP Gases),4/17/00,4/17/00,4/17/00 22:08,4/17/00 22:09,4/17/00 22:09,...,10,10,Bayview Hunters Point,"(37.7262373966555, -122.393902508468)",001080336-E17,37.726237,-122.393903,"(-122.393902508468, 37.7262373966555)",POINT (-122.393902508468 37.7262373966555),616.0
4,4,1090191,E40,32517,Gas Leak (Natural and LP Gases),4/18/00,4/18/00,4/18/00 16:44,4/18/00 16:45,4/18/00 16:46,...,8,4,Sunset/Parkside,"(37.7617311116621, -122.47754025709)",001090191-E40,37.761731,-122.477540,"(-122.47754025709, 37.7617311116621)",POINT (-122.47754025709 37.7617311116621),194.0


In [5]:
df.shape

(17312, 40)

In [6]:
df['datetime'] = pd.to_datetime(df['Received DtTm'])

In [7]:
df['year'] = df['datetime'].dt.year
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['week'] = df['datetime'].dt.weekday

In [8]:
df['week2'] = df['week'].apply(lambda x: int(x)/5)

In [9]:
df['month2'] = (df['year']-2000)*12+df['month']

In [10]:
df['season'] = df['month'].apply(lambda x: int(x)/3)
df['season'] = df['season'].replace(0, 4)

In [11]:
df['Lat'] = df.Location.apply(lambda x: float(x.strip( '()' ).split(",")[0]))
df['Lon'] = df.Location.apply(lambda x: float(x.strip( '()' ).split(",")[1]))

In [12]:
df = df[['Call Number','Incident Number','datetime','year','hour','day','month','week','season','Lat','Lon','FID']]

In [13]:
df.head()

,Call Number,Incident Number,datetime,year,hour,day,month,week,season,Lat,Lon,FID
0,1050242,31236,2000-04-14 16:58:00,2000,16,14,4,4,1,37.763317,-122.458206,286.0
1,1050319,31311,2000-04-14 20:34:00,2000,20,14,4,4,1,37.709603,-122.409889,551.0
2,1070275,31946,2000-04-16 20:38:00,2000,20,16,4,6,1,37.729194,-122.374002,731.0
3,1080336,32305,2000-04-17 22:08:00,2000,22,17,4,0,1,37.726237,-122.393903,616.0
4,1090191,32517,2000-04-18 16:44:00,2000,16,18,4,1,1,37.761731,-122.477540,194.0


In [14]:
df.shape

(17312, 12)

In [15]:
len(df['FID'].unique())

554

In [16]:
df = df[~((df['Lat'] > 37.807070) & (df['Lon'] > -122.379489))]
df.shape

(17153, 12)

In [17]:
#dorp duplicate
df['Incident Number'].unique().shape
df = df.drop_duplicates('Incident Number')
df.reset_index(inplace=True, drop=True) 
df.head()

,Call Number,Incident Number,datetime,year,hour,day,month,week,season,Lat,Lon,FID
0,1050242,31236,2000-04-14 16:58:00,2000,16,14,4,4,1,37.763317,-122.458206,286.0
1,1050319,31311,2000-04-14 20:34:00,2000,20,14,4,4,1,37.709603,-122.409889,551.0
2,1070275,31946,2000-04-16 20:38:00,2000,20,16,4,6,1,37.729194,-122.374002,731.0
3,1080336,32305,2000-04-17 22:08:00,2000,22,17,4,0,1,37.726237,-122.393903,616.0
4,1090191,32517,2000-04-18 16:44:00,2000,16,18,4,1,1,37.761731,-122.477540,194.0


In [18]:
df.shape

(8526, 12)

In [19]:
len(df['FID'].unique())

553

In [20]:
df[df['FID'].isnull()]

,Call Number,Incident Number,datetime,year,hour,day,month,week,season,Lat,Lon,FID
1309,32900228,3083237,2003-10-17 14:22:00,2003,14,17,10,4,3,37.700091,-122.414099,NaN
2542,70570351,7016778,2007-02-26 21:47:00,2007,21,26,2,0,4,37.703080,-122.454856,NaN
2870,80390418,8012455,2008-02-08 20:22:00,2008,20,8,2,4,4,37.704881,-122.408989,NaN
4205,110580134,11019046,2011-02-27 09:57:00,2011,9,27,2,6,4,37.706248,-122.413350,NaN
4435,112550063,11084038,2011-09-12 08:19:00,2011,8,12,9,0,3,37.707091,-122.449290,NaN


In [21]:
df = df[~(df['FID'].isnull())]

In [22]:
df['FID'] = df['FID'].astype(int)

In [23]:
df['Row'] = df['FID']%23 +1
df['Col'] = df['FID']/23 +1 

In [24]:
df.head()

,Call Number,Incident Number,datetime,year,hour,day,month,week,season,Lat,Lon,FID,Row,Col
0,1050242,31236,2000-04-14 16:58:00,2000,16,14,4,4,1,37.763317,-122.458206,286,11,13.434783
1,1050319,31311,2000-04-14 20:34:00,2000,20,14,4,4,1,37.709603,-122.409889,551,23,24.956522
2,1070275,31946,2000-04-16 20:38:00,2000,20,16,4,6,1,37.729194,-122.374002,731,19,32.782609
3,1080336,32305,2000-04-17 22:08:00,2000,22,17,4,0,1,37.726237,-122.393903,616,19,27.782609
4,1090191,32517,2000-04-18 16:44:00,2000,16,18,4,1,1,37.761731,-122.477540,194,11,9.434783


In [25]:
df['Col'] = df['Col'].apply(lambda x: int(x))

In [26]:
#split by year
df2010 = df[df['year'] >= 2010]  #df after 2010

In [27]:
df2000 = df[df['year'] < 2010]  #df before 2010

In [28]:
df2010.shape

(4888, 14)

In [29]:
df2000.shape

(3633, 14)

In [30]:
df2010_anodf = df2010.groupby(['Row','Col']).count()
#df_ = df.groupby(['FID','month']).size().reset_index()
df2010_anodf.head(2)

Call Number  Incident Number  datetime  year  hour  day  month  week  \
Row Col                                                                         
1   9              2                2         2     2     2    2      2     2   
    10             1                1         1     1     1    1      1     1   

         season  Lat  Lon  FID  
Row Col                         
1   9         2    2    2    2  
    10        1    1    1    1

In [31]:
df2010_anodf.reset_index(inplace=True)

In [32]:
df2010_anodf = df2010_anodf.iloc[:,:3]

In [34]:
addf_fid = df.groupby(['FID']).size().reset_index()

In [35]:
addf_fid.columns = ['FID','NumEvents']
addf_fid.head()

,FID,NumEvents
0,7,3
1,8,13
2,9,1
3,10,3
4,29,2


In [36]:
len(addf_fid.FID.unique())

552

In [37]:
df_ = df.groupby(['FID','month']).size().reset_index()
df_.columns = ['FID','month','NumEvents']

In [38]:
df_.shape

(3877, 3)

In [39]:
df_['NumEvents'].describe()

count    3877.000000
mean        2.197833
std         1.652254
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max        15.000000
Name: NumEvents, dtype: float64

In [42]:
#match feature names
name = pd.read_csv('name_dictionary.csv')
name

,Unnamed: 0,aland10,awater10,countyfp10,tractce10,mtfcc10,name10,namelsad10,intptlon10,funcstat10,...,Estimate; Renter-occupied housing units: - High school graduate (including equivalency)_education,Estimate; Renter-occupied housing units: - Some college or associate's degree_education,Estimate; Renter-occupied housing units: - Bachelor's degree or higher_education,Estimate; Total:_tenure_kitchen,Estimate; Owner occupied:_tenure_kitchen,Estimate; Owner occupied: - Complete kitchen facilities_tenure_kitchen,Estimate; Owner occupied: - Lacking complete kitchen facilities_tenure_kitchen,Estimate; Renter occupied:_tenure_kitchen,Estimate; Renter occupied: - Complete kitchen facilities_tenure_kitchen,Estimate; Renter occupied: - Lacking complete kitchen facilities_tenure_kitchen
0,0,370459.0,0.0,75,16500,G5020,165.00,Census Tract 165,-122.447788,S,...,91.0,439.0,1273.0,2370.0,515.0,515.0,0.0,1855.0,1841.0,14.0
1,1,309097.0,0.0,75,16400,G5020,164.00,Census Tract 164,-122.436973,S,...,120.0,310.0,1062.0,1873.0,334.0,334.0,0.0,1539.0,1539.0,0.0
2,2,245867.0,0.0,75,16300,G5020,163.00,Census Tract 163,-122.429551,S,...,232.0,311.0,999.0,1971.0,377.0,377.0,0.0,1594.0,1473.0,121.0
3,3,368901.0,0.0,75,16100,G5020,161.00,Census Tract 161,-122.428663,S,...,656.0,471.0,886.0,2680.0,175.0,175.0,0.0,2505.0,2475.0,30.0
4,4,158236.0,0.0,75,16000,G5020,160.00,Census Tract 160,-122.422484,S,...,146.0,271.0,581.0,1578.0,443.0,443.0,0.0,1135.0,1108.0,27.0
5,5,295388.0,0.0,75,15900,G5020,159.00,Census Tract 159,-122.429343,S,...,59.0,739.0,909.0,2315.0,481.0,481.0,0.0,1834.0,1776.0,58.0
6,6,836550.0,0.0,75,15700,G5020,157.00,Census Tract 157,-122.448007,S,...,180.0,383.0,1153.0,2483.0,688.0,624.0,64.0,1795.0,1795.0,0.0
7,7,293492.0,0.0,75,15600,G5020,156.00,Census Tract 156,-122.456425,S,...,81.0,203.0,552.0,1288.0,449.0,449.0,0.0,839.0,839.0,0.0
8,8,312624.0,0.0,75,15500,G5020,155.00,Census Tract 155,-122.434135,S,...,232.0,486.0,1208.0,2335.0,202.0,187.0,15.0,2133.0,1915.0,218.0
9,9,719856.0,0.0,75,15400,G5020,154.00,Census Tract 154,-122.451036,S,...,78.0,346.0,1590.0,2913.0,804.0,804.0,0.0,2109.0,2062.0,47.0


In [43]:
name.shape

(197, 86)

In [44]:
name.iloc[:,67].head(2)  # total income

0    104083.0
1    100859.0
Name: Estimate; Total:_income, dtype: float64

In [321]:
sfshp.iloc[:,3:-2].loc[:,name.columns[14::].map(lambda x:'income' in x).values]

,meanEsti53
0,140556.00000
1,111806.00000
2,111806.00000
3,83056.00000
4,76480.00000
5,76480.00000
6,78659.00000
7,83036.50000
8,89395.46907
9,95734.93814


In [213]:
features.iloc[:,65:66] # total income

,Estimate53
0,104083.000000
1,100859.000000
2,50714.000000
3,32411.000000
4,100705.000000
5,49679.000000
6,132589.000000
7,103828.000000
8,46452.000000
9,116983.000000


In [46]:
features = gpd.GeoDataFrame.from_file('San_Francisco_shape_file/sf.shp')
features

,aland10,awater10,countyfp10,tractce10,mtfcc10,name10,namelsad10,intptlon10,funcstat10,intptlat10,...,Estimate63,Estimate64,Estimate65,Estimate66,Estimate67,Estimate68,Estimate69,Estimate70,Estimate71,geometry
0,370459.0,0.0,075,016500,G5020,165,Census Tract 165,-122.4477884,S,+37.7741958,...,439,1273,2370,515,515,0,1855,1841,14,"POLYGON ((-122.446471 37.775802, -122.44478 37..."
1,309097.0,0.0,075,016400,G5020,164,Census Tract 164,-122.4369729,S,+37.7750995,...,310,1062,1873,334,334,0,1539,1539,0,"POLYGON ((-122.44034 37.77658, -122.439844 37...."
2,245867.0,0.0,075,016300,G5020,163,Census Tract 163,-122.4295509,S,+37.7760456,...,311,999,1971,377,377,0,1594,1473,121,"POLYGON ((-122.429152 37.778007, -122.428909 3..."
3,368901.0,0.0,075,016100,G5020,161,Census Tract 161,-122.4286631,S,+37.7799831,...,471,886,2680,175,175,0,2505,2475,30,"POLYGON ((-122.428909 37.778039, -122.429152 3..."
4,158236.0,0.0,075,016000,G5020,160,Census Tract 160,-122.4224838,S,+37.7823363,...,271,581,1578,443,443,0,1135,1108,27,"POLYGON ((-122.420425 37.780583, -122.420336 3..."
5,295388.0,0.0,075,015900,G5020,159,Census Tract 159,-122.4293428,S,+37.7833400,...,739,909,2315,481,481,0,1834,1776,58,"POLYGON ((-122.432329 37.781428, -122.433251 3..."
6,836550.0,0.0,075,015700,G5020,157,Census Tract 157,-122.4480069,S,+37.7789154,...,383,1153,2483,688,624,64,1795,1795,0,"POLYGON ((-122.441678 37.777373, -122.441634 3..."
7,293492.0,0.0,075,015600,G5020,156,Census Tract 156,-122.4564254,S,+37.7782427,...,203,552,1288,449,449,0,839,839,0,"POLYGON ((-122.454683 37.774755, -122.455253 3..."
8,312624.0,0.0,075,015500,G5020,155,Census Tract 155,-122.4341350,S,+37.7850923,...,486,1208,2335,202,187,15,2133,1915,218,"POLYGON ((-122.432914 37.784355, -122.43297 37..."
9,719856.0,0.0,075,015400,G5020,154,Census Tract 154,-122.4510358,S,+37.7843608,...,346,1590,2913,804,804,0,2109,2062,47,"POLYGON ((-122.44337 37.785759, -122.443183 37..."


In [47]:
features.iloc[:,13:22] # year buit

,Estimate_1,Estimate_2,Estimate_3,Estimate_4,Estimate_5,Estimate_6,Estimate_7,Estimate_8,Estimate_9
0,0,37,56,132,248,164,84,42,1770
1,2,83,46,91,41,40,148,163,1354
2,0,0,283,95,112,161,122,61,1323
3,0,141,321,673,691,342,208,173,409
4,0,147,136,440,172,412,29,60,368
5,0,160,663,518,297,357,98,44,223
6,18,259,122,51,301,274,413,374,934
7,19,0,50,28,111,155,149,185,654
8,0,65,220,505,486,394,155,150,525
9,0,16,48,106,445,164,349,260,1681


In [52]:
name.columns[14::].map(lambda x:'year' in x).values

array([True, True, True, True, True, True, True, True, True, True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False], dtype=object)

In [48]:
name.iloc[:,15:25].head(2)# year built

,Estimate; Total: - Built 2010 or later_year,Estimate; Total: - Built 2000 to 2009_year,Estimate; Total: - Built 1990 to 1999_year,Estimate; Total: - Built 1980 to 1989_year,Estimate; Total: - Built 1970 to 1979_year,Estimate; Total: - Built 1960 to 1969_year,Estimate; Total: - Built 1950 to 1959_year,Estimate; Total: - Built 1940 to 1949_year,Estimate; Total: - Built 1939 or earlier_year,Estimate; Total:_rent
0,0.0,37.0,56.0,132.0,248.0,164.0,84.0,42.0,1770.0,1855.0
1,2.0,83.0,46.0,91.0,41.0,40.0,148.0,163.0,1354.0,1539.0


In [146]:
features.columns

Index([   u'aland10',   u'awater10', u'countyfp10',  u'tractce10',
          u'mtfcc10',     u'name10', u'namelsad10', u'intptlon10',
       u'funcstat10', u'intptlat10',  u'statefp10',    u'geoid10',
        u'Estimate;', u'Estimate_1', u'Estimate_2', u'Estimate_3',
       u'Estimate_4', u'Estimate_5', u'Estimate_6', u'Estimate_7',
       u'Estimate_8', u'Estimate_9', u'Estimate10', u'Estimate11',
       u'Estimate12', u'Estimate13', u'Estimate14', u'Estimate15',
       u'Estimate16', u'Estimate17', u'Estimate18', u'Estimate19',
       u'Estimate20', u'Estimate21', u'Estimate22', u'Estimate23',
       u'Estimate24', u'Estimate25', u'Estimate26', u'Estimate27',
       u'Estimate28', u'Estimate29', u'Estimate30', u'Estimate31',
       u'Estimate32', u'Estimate33', u'Estimate34', u'Estimate35',
       u'Estimate36', u'Estimate37', u'Estimate38', u'Estimate39',
       u'Estimate40', u'Estimate41', u'Estimate42', u'Estimate43',
       u'Estimate44', u'Estimate45', u'Estimate46', u'Estimate

In [50]:
sfshp = gpd.GeoDataFrame.from_file('SF_grid_features/SF_gird_features.shp')
sfshp

,FID,meanaland1,meanawater,meanEstima,meanEsti_1,meanEsti_2,meanEsti_3,meanEsti_4,meanEsti_5,meanEsti_6,...,meanEsti64,meanEsti65,meanEsti66,meanEsti67,meanEsti68,meanEsti69,meanEsti70,meanEsti71,count,geometry
0,0,6.108838e+06,2.031402e+07,1280.00000,0.0,0.00000,7.00000,0.00000,218.00000,272.00000,...,891.00000,1092.00000,0.00000,0.00000,0.00000,1092.00000,1050.00000,42.00000,1.0,"POLYGON ((-122.51364836 37.811165, -122.509148..."
1,1,3.637716e+06,1.220504e+07,2041.00000,0.0,38.50000,55.50000,113.00000,161.00000,306.50000,...,801.50000,1872.50000,681.00000,681.00000,0.00000,1191.50000,1147.50000,44.00000,2.0,"POLYGON ((-122.51364836 37.806665, -122.509148..."
2,2,3.637716e+06,1.220504e+07,2041.00000,0.0,38.50000,55.50000,113.00000,161.00000,306.50000,...,801.50000,1872.50000,681.00000,681.00000,0.00000,1191.50000,1147.50000,44.00000,2.0,"POLYGON ((-122.51364836 37.802165, -122.509148..."
3,3,1.166594e+06,4.096057e+06,2802.00000,0.0,77.00000,104.00000,226.00000,104.00000,341.00000,...,712.00000,2653.00000,1362.00000,1362.00000,0.00000,1291.00000,1245.00000,46.00000,1.0,"POLYGON ((-122.51364836 37.797665, -122.509148..."
4,4,1.094580e+06,2.437548e+06,1446.50000,0.0,43.50000,52.00000,113.00000,52.00000,175.00000,...,373.00000,1372.00000,700.00000,700.00000,0.00000,672.00000,649.00000,23.00000,2.0,"POLYGON ((-122.51364836 37.793165, -122.509148..."
5,5,1.094580e+06,2.437548e+06,1446.50000,0.0,43.50000,52.00000,113.00000,52.00000,175.00000,...,373.00000,1372.00000,700.00000,700.00000,0.00000,672.00000,649.00000,23.00000,2.0,"POLYGON ((-122.51364836 37.788665, -122.509148..."
6,6,8.623000e+05,1.625032e+06,1514.00000,0.0,37.33333,38.66667,101.00000,70.00000,157.00000,...,400.66667,1436.33333,734.00000,734.00000,0.00000,702.33333,687.00000,15.33333,3.0,"POLYGON ((-122.51364836 37.784165, -122.509148..."
7,7,7.821665e+05,2.048028e+06,2225.50000,0.0,51.00000,58.00000,151.50000,105.00000,231.00000,...,584.00000,2109.00000,1082.00000,1082.00000,0.00000,1027.00000,1004.00000,23.00000,2.0,"POLYGON ((-122.51364836 37.779665, -122.509148..."
8,8,2.755932e+06,2.252717e+06,1414.50000,0.0,38.50000,52.00000,113.00000,52.00000,176.50000,...,369.50000,1340.00000,681.00000,681.00000,0.00000,659.00000,636.00000,23.00000,2.0,"POLYGON ((-122.51364836 37.775165, -122.509148..."
9,9,4.345271e+06,4.093770e+05,27.00000,0.0,0.00000,0.00000,0.00000,0.00000,12.00000,...,27.00000,27.00000,0.00000,0.00000,0.00000,27.00000,27.00000,0.00000,1.0,"POLYGON ((-122.51364836 37.770665, -122.509148..."


In [53]:
sfshp.iloc[:,4:13].head(2)# meanEsti11,meanEsti12 = Estimate; No bedroom:_rent,Estimate; No bedroom: - With cash rent:_rent

,meanEsti_1,meanEsti_2,meanEsti_3,meanEsti_4,meanEsti_5,meanEsti_6,meanEsti_7,meanEsti_8,meanEsti_9
0,0.0,0.0,7.0,0.0,218.0,272.0,196.0,214.0,373.0
1,0.0,38.5,55.5,113.0,161.0,306.5,241.5,302.0,823.0


In [54]:
sfshp_ano_features = sfshp.iloc[:,4:13]

In [55]:
sfshp_ano_features['income'] = sfshp.iloc[:,56:57]

In [57]:
sfshp.iloc[:,3:-2].loc[:,name.columns[14::].map(lambda x:'year' in x).values]

,meanEstima,meanEsti_1,meanEsti_2,meanEsti_3,meanEsti_4,meanEsti_5,meanEsti_6,meanEsti_7,meanEsti_8,meanEsti_9
0,1280.00000,0.0,0.00000,7.00000,0.00000,218.00000,272.00000,196.00000,214.00000,373.00000
1,2041.00000,0.0,38.50000,55.50000,113.00000,161.00000,306.50000,241.50000,302.00000,823.00000
2,2041.00000,0.0,38.50000,55.50000,113.00000,161.00000,306.50000,241.50000,302.00000,823.00000
3,2802.00000,0.0,77.00000,104.00000,226.00000,104.00000,341.00000,287.00000,390.00000,1273.00000
4,1446.50000,0.0,43.50000,52.00000,113.00000,52.00000,175.00000,145.00000,213.50000,652.50000
5,1446.50000,0.0,43.50000,52.00000,113.00000,52.00000,175.00000,145.00000,213.50000,652.50000
6,1514.00000,0.0,37.33333,38.66667,101.00000,70.00000,157.00000,141.00000,263.33333,705.66667
7,2225.50000,0.0,51.00000,58.00000,151.50000,105.00000,231.00000,210.00000,376.50000,1042.50000
8,1414.50000,0.0,38.50000,52.00000,113.00000,52.00000,176.50000,143.50000,195.00000,644.00000
9,27.00000,0.0,0.00000,0.00000,0.00000,0.00000,12.00000,0.00000,0.00000,15.00000


In [58]:
name.columns.map(lambda x: 'year' in x)

Index([False, False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False],
      dtype='object')

## anomaly detection

In [59]:
sfshp_ano_features.head()

,meanEsti_1,meanEsti_2,meanEsti_3,meanEsti_4,meanEsti_5,meanEsti_6,meanEsti_7,meanEsti_8,meanEsti_9,income
0,0.0,0.0,7.0,0.0,218.0,272.0,196.0,214.0,373.0,140556.0
1,0.0,38.5,55.5,113.0,161.0,306.5,241.5,302.0,823.0,111806.0
2,0.0,38.5,55.5,113.0,161.0,306.5,241.5,302.0,823.0,111806.0
3,0.0,77.0,104.0,226.0,104.0,341.0,287.0,390.0,1273.0,83056.0
4,0.0,43.5,52.0,113.0,52.0,175.0,145.0,213.5,652.5,76480.0


In [61]:
sfshp_ano_features['FID'] = sfshp.FID

In [62]:
ADdf = addf_fid.merge(sfshp_ano_features,on='FID',how='inner')

In [63]:
ADdf.columns = ['FID','Num','Built10orlater','Built00to09','Built90to99','Built80to89','Built70to79','Built60to69','Built50to59','Built40to49','Built39orEarlier','INCOME']

In [64]:
ADdf.head()

,FID,Num,Built10orlater,Built00to09,Built90to99,Built80to89,Built70to79,Built60to69,Built50to59,Built40to49,Built39orEarlier,INCOME
0,7,3,0.0,51.00000,58.00000,151.50000,105.00000,231.00000,210.00000,376.50000,1042.50000,83036.50000
1,8,13,0.0,38.50000,52.00000,113.00000,52.00000,176.50000,143.50000,195.00000,644.00000,89395.46907
2,9,1,0.0,0.00000,0.00000,0.00000,0.00000,12.00000,0.00000,0.00000,15.00000,95734.93814
3,10,3,0.0,34.00000,38.66667,22.66667,135.66667,154.33333,217.66667,348.66667,345.00000,81661.31271
4,29,2,0.0,37.33333,38.66667,101.00000,70.00000,157.00000,141.00000,263.33333,705.66667,78659.00000


In [65]:
ADdf.columns

Index([u'FID', u'Num', u'Built10orlater', u'Built00to09', u'Built90to99',
       u'Built80to89', u'Built70to79', u'Built60to69', u'Built50to59',
       u'Built40to49', u'Built39orEarlier', u'INCOME'],
      dtype='object')

### LocalOutlierFactor

In [67]:
ADdf.head(2)

,FID,Num,Built10orlater,Built00to09,Built90to99,Built80to89,Built70to79,Built60to69,Built50to59,Built40to49,Built39orEarlier,INCOME
0,7,3,0.0,51.0,58.0,151.5,105.0,231.0,210.0,376.5,1042.5,83036.50000
1,8,13,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,89395.46907


In [68]:
X = ADdf.iloc[:,1:]

In [69]:
X.head(2)

,Num,Built10orlater,Built00to09,Built90to99,Built80to89,Built70to79,Built60to69,Built50to59,Built40to49,Built39orEarlier,INCOME
0,3,0.0,51.0,58.0,151.5,105.0,231.0,210.0,376.5,1042.5,83036.50000
1,13,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,89395.46907


In [70]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=20)
clf.fit_predict(X)

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [71]:
score = pd.DataFrame(clf._decision_function(X))
score = pd.concat((ADdf.iloc[:,0],score), axis = 1)
score.columns = ['FID','Anomaly score']

In [72]:
score

,FID,Anomaly score
0,7,-1.013337
1,8,-0.976877
2,9,-0.988370
3,10,-0.985805
4,29,-1.023840
5,30,-1.013337
6,31,-0.976877
7,33,-0.993047
8,34,-0.956790
9,35,-1.003499


In [73]:
score.sort_values(by='Anomaly score')[:10]

,FID,Anomaly score
321,425,-3.895669
40,97,-2.415838
196,293,-2.319325
322,426,-2.305691
270,372,-2.302885
397,503,-2.266459
68,143,-2.240797
248,349,-2.189454
547,707,-2.082862
375,481,-2.040166


In [74]:
#TOP 5 anomalous FID
ADdf[(ADdf['FID'] == 425) | 
     (ADdf['FID'] == 97)|
     (ADdf['FID'] == 293)|
     (ADdf['FID'] == 426)|
     (ADdf['FID'] == 372)
#      (ADdf['FID'] == 503)|
#      (ADdf['FID'] == 143)|
#      (ADdf['FID'] == 349)|
#      (ADdf['FID'] == 707)|
#      (ADdf['FID'] == 481)
    ]

,FID,Num,Built10orlater,Built00to09,Built90to99,Built80to89,Built70to79,Built60to69,Built50to59,Built40to49,Built39orEarlier,INCOME
40,97,1,0.00,22.5,2.00,2.50,13.0,18.50,13.5,81.0,401.00,142483.50
196,293,4,4.25,14.5,55.75,77.75,79.5,162.75,374.0,304.0,1156.75,142350.25
270,372,13,0.00,32.5,24.50,62.00,179.5,204.00,139.5,149.0,1648.00,212517.50
321,425,19,0.00,27.0,49.00,50.00,302.0,198.00,208.0,135.0,1872.00,243800.00
322,426,15,0.00,40.5,49.50,66.00,203.5,145.50,150.5,102.0,1672.50,212577.00


In [75]:
ADdf.describe()

,FID,Num,Built10orlater,Built00to09,Built90to99,Built80to89,Built70to79,Built60to69,Built50to59,Built40to49,Built39orEarlier,INCOME
count,552.000000,552.000000,552.000000,552.000000,552.000000,552.000000,552.000000,552.000000,552.000000,552.000000,552.000000,552.000000
mean,371.125000,15.436594,3.326475,214.725738,95.931699,102.568931,143.468293,151.863186,194.574435,233.815847,766.009554,104471.567402
std,179.879504,14.855980,7.127712,611.972301,180.888548,148.505917,132.304842,101.320053,130.611302,220.166479,481.167408,37410.335228
min,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19816.000000
25%,225.750000,6.000000,0.000000,22.250000,29.000000,39.000000,61.500000,82.187500,110.875000,103.249998,373.000000,76511.125000
50%,377.500000,10.000000,0.000000,44.550000,49.583335,67.625000,102.000000,138.583335,177.750000,172.250000,713.833335,96274.125000
75%,519.250000,21.000000,4.000000,93.000000,89.062500,108.666670,188.874998,201.000000,246.333332,293.499998,1091.550003,134544.083333
max,756.000000,122.000000,52.000000,4011.000000,1734.000000,1371.000000,824.000000,643.000000,920.000000,1425.000000,2089.000000,243800.000000


In [352]:
df_FID = df.groupby(['FID']).count().reset_index()

In [355]:
df_FID = df_FID.iloc[:,:2]

In [356]:
df_FID.head()

,FID,Call Number
0,7,3
1,8,13
2,9,1
3,10,3
4,29,2


## decision tree

In [350]:
len(df_.FID.unique())

533

In [374]:
sfshp1 = sfshp.merge(df_,on='FID',how='inner')

In [379]:
len(sfshp1.FID.unique())

533

In [35]:
sfshp1.shape

(2856, 79)

In [36]:
sfshp1['Row'] = sfshp1['FID']%23 +1
sfshp1['Col'] = sfshp1['FID']/23 +1 
sfshp1['Col'] = sfshp1['Col'].apply(lambda x: int(x))

In [37]:
sfshp1.columns

Index([       u'FID', u'meanaland1', u'meanawater', u'meanEstima',
       u'meanEsti_1', u'meanEsti_2', u'meanEsti_3', u'meanEsti_4',
       u'meanEsti_5', u'meanEsti_6', u'meanEsti_7', u'meanEsti_8',
       u'meanEsti_9', u'meanEsti10', u'meanEsti11', u'meanEsti12',
       u'meanEsti13', u'meanEsti14', u'meanEsti15', u'meanEsti16',
       u'meanEsti17', u'meanEsti18', u'meanEsti19', u'meanEsti20',
       u'meanEsti21', u'meanEsti22', u'meanEsti23', u'meanEsti24',
       u'meanEsti25', u'meanEsti26', u'meanEsti27', u'meanEsti28',
       u'meanEsti29', u'meanEsti30', u'meanEsti31', u'meanEsti32',
       u'meanEsti33', u'meanEsti34', u'meanEsti35', u'meanEsti36',
       u'meanEsti37', u'meanEsti38', u'meanEsti39', u'meanEsti40',
       u'meanEsti41', u'meanEsti42', u'meanEsti43', u'meanEsti44',
       u'meanEsti45', u'meanEsti46', u'meanEsti47', u'meanEsti48',
       u'meanEsti49', u'meanEsti50', u'meanEsti51', u'meanEsti52',
       u'meanEsti53', u'meanEsti54', u'meanEsti55', u'meanEsti

In [38]:
sfshp1.drop(['FID','meanaland1','meanawater', 'count', 'geometry'], 
        axis=1, inplace=True)

In [39]:
sfshp1.head()

,meanEstima,meanEsti_1,meanEsti_2,meanEsti_3,meanEsti_4,meanEsti_5,meanEsti_6,meanEsti_7,meanEsti_8,meanEsti_9,...,meanEsti66,meanEsti67,meanEsti68,meanEsti69,meanEsti70,meanEsti71,month,NumEvents,Row,Col
0,2225.5,0.0,51.0,58.0,151.5,105.0,231.0,210.0,376.5,1042.5,...,1082.0,1082.0,0.0,1027.0,1004.0,23.0,4,1,8,1
1,1414.5,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,...,681.0,681.0,0.0,659.0,636.0,23.0,2,1,9,1
2,1414.5,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,...,681.0,681.0,0.0,659.0,636.0,23.0,4,1,9,1
3,1414.5,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,...,681.0,681.0,0.0,659.0,636.0,23.0,6,2,9,1
4,1414.5,0.0,38.5,52.0,113.0,52.0,176.5,143.5,195.0,644.0,...,681.0,681.0,0.0,659.0,636.0,23.0,7,1,9,1


In [40]:
from sklearn.model_selection import train_test_split
# Let's take "found.weapon" as the target variable. 
y=sfshp1.loc[:,"NumEvents"]

# Get the feature space.  We are using only features from before the stop, getting rid of features from during/after the stop like "arrested".
X=pd.concat([sfshp1.loc[:,"meanEsti_1":"month"], sfshp1.loc[:,["Row","Col"]]],axis=1)
#X=pd.concat([sfshp1.loc[:,["Row","Col"]]],axis=1)
#X=pd.concat([sfshp1.loc[:,"season"], sfshp1.loc[:,["Row","Col"]]],axis=1)
# Split data into 70% train, 30% test
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3, random_state=999)
print X_train.head()

      meanEsti_1  meanEsti_2  meanEsti_3  meanEsti_4  meanEsti_5  meanEsti_6  \
1207         0.0    25.60000    48.80000    64.80000   166.40000   198.00000   
2390         0.0   711.33333   310.66667   305.66667   192.66667    65.33333   
2522         0.0   217.33333   131.66667   193.33333   130.33333    71.00000   
2021         0.0    99.50000   138.25000   169.50000   184.50000   280.25000   
166          0.0   140.00000    91.00000    70.00000   824.00000    86.00000   

      meanEsti_7  meanEsti_8  meanEsti_9  meanEsti10 ...   meanEsti65  \
1207   116.40000   126.80000  1316.40000  1289.00000 ...   1845.20000   
2390    65.00000    56.00000   365.66667  1238.00000 ...   1751.00000   
2522    78.33333   162.33333   791.33333   930.66667 ...   1617.33333   
2021   219.75000    78.50000  1427.00000  1877.25000 ...   2316.75000   
166     73.00000     0.00000    19.00000  1070.00000 ...   1276.00000   

      meanEsti66  meanEsti67  meanEsti68  meanEsti69  meanEsti70  meanEsti71  \


In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# learn model
#dt=DecisionTreeClassifier(max_depth=10)
dt=DecisionTreeClassifier(max_leaf_nodes=20)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
accuracy_class = metrics.accuracy_score(y_test,y_pred)
error_mean = metrics.mean_squared_error(y_test,y_pred)
print "MSE: ", error_mean

# in sample accuracy
print 'In sample accuracy:',dt.score(X_train,y_train)

# out of sample accuracy
print 'Out of sample accuracy:',dt.score(X_test,y_test)

MSE:  1.42823803967
In sample accuracy: 0.610805402701
Out of sample accuracy: 0.556592765461


In [42]:
# use a single train/test split
OS=[]
for i in range(2,10,1):
    dt = DecisionTreeClassifier(max_depth=i)
    dt.fit(X_train,y_train)
    OS.append(dt.score(X_test,y_test))

In [43]:
OS

[0.59393232205367563,
 0.57992998833138854,
 0.57992998833138854,
 0.59043173862310383,
 0.54375729288214703,
 0.54259043173862309,
 0.52742123687281217,
 0.5250875145857643]

## plot the tree

In [46]:
import io
import pydotplus
import numpy as np
import pandas as pd
import sklearn.datasets as datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from IPython.display import Image 

import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [65]:
# Train model
dtree = DecisionTreeRegressor(max_depth=4)
dtree.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [61]:
import StringIO

In [74]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())
graph.write_png('DT.png')#save the figure

True